<a href="https://colab.research.google.com/github/sanskriti49/spam-detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# importing libraries
import numpy as py
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importing Dataset

In [19]:
url="https://raw.githubusercontent.com/sanskriti49/spam-detection/main/spam.csv"
df=pd.read_csv(url, encoding="ISO-8859-1")
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


## Label Encoding

In [20]:
df = df.rename(columns={'v1': 'label', 'v2': 'message'})

# encode the labels: 'ham' means 0 , 'spam' means 1
df['label']=df['label'].map({'ham':0, 'spam':1})

x=df['message']
y=df['label']

## Data Pre-processing

In [33]:
ps=PorterStemmer()
corpus=[]

y=y.map({1:"Spam",0:"Not spam"})

for i in range(0,len(x)):
  email=re.sub('[^a-zA-Z]',' ',str(x.iloc[i]))
  email=email.lower()
  email=email.split(' ')

  email=[ps.stem(word) for word in email if word not in stopwords.words('english')]
  email=' '.join(email)
  corpus.append(email)



<ipython-input-33-8cc310900cef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]="Not spam"
<ipython-input-33-8cc310900cef>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not spam' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y[i]="Not spam"


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

## Data transformation

In [34]:
# CountVectorizer matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(corpus).toarray()

ValueError: empty vocabulary; perhaps the documents only contain stop words

## Train-Test split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
# applying MultiNomialNB model
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB().fit(x_train, y_train)

## Making predictions and creating a Confusion Matrix

In [ ]:
y_pred=classifier.predict(x_test)

# creating confusion matrix
# from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_pred)
print(confusion_m)

import seaborn as sns
import matplotlib.pyplot as plt

# Confusion Matrix Heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_m, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix Heatmap')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Getting the accuracy

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

## Dumping Models

In [ ]:
import pickle
pickle.dump(classifier,open('model.pkl','wb'))
pickle.dump(cv,open('cv-model.pkl','wb'))

final_df = pd.DataFrame({'Subject': df['message'], 'Label': y})
print(final_df.head())